In [153]:
import ast
from typing import List
from os import listdir
import spotipy.util as util
import spotipy
import requests
import pandas as pd

In [154]:
def get_streamings(path: str = 'MyData') -> List[dict]:
    
    files = ['MyData/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [155]:
username = 'dsketron'
client_id ='dd4ec62f1010410cb5c44350cb5c79b2'
client_secret = '92213c6ac1a44b3fad71c1bd32fae3b2'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [156]:
print(token)

BQBiZm2rql-yD_a3P0mqNmWKkTWzVoowOOBExqgCHiIhS_glKLRg3A_mEnxhD-t5iDLuAfgIhUvnryzCkfTRA3vTV7WZTfosoXGJ11Oesj2iG6ydPjaAWwpB3q5eh0J-x8aiugKLwgGo6OtbqZTG


In [157]:
{"access_token": "BQDsr-yBfBfIjesdN_G7sD-XBz9CNMWwr9J3jukc4IwRUMJyWojrgtmWL3nXOyBn2-VytfVSqr_Xpc_naxUpyA-zEcuGuq8Jl0k6wKKz0RI99pUwvZx6JfV6aJXGm_VilIjwBxUQibG5zLAnTC5z", 
"token_type": "Bearer", "expires_in": 3600,
"refresh_token": "AQCeb3UPba7uld71D8MnommPk63Ddlk2r8fENXIx9Y5TLkhPOESu2rrFQs8IDtRVFO_FEdcfhi5GrHVqpuY3fR-N9ny__-ku8BIOzFEpDytCVN6Iw9rrtTnGvbNFG6fEarM", 
"scope": "user-read-recently-played", 
"expires_at": 1590359901}

{'access_token': 'BQDsr-yBfBfIjesdN_G7sD-XBz9CNMWwr9J3jukc4IwRUMJyWojrgtmWL3nXOyBn2-VytfVSqr_Xpc_naxUpyA-zEcuGuq8Jl0k6wKKz0RI99pUwvZx6JfV6aJXGm_VilIjwBxUQibG5zLAnTC5z',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQCeb3UPba7uld71D8MnommPk63Ddlk2r8fENXIx9Y5TLkhPOESu2rrFQs8IDtRVFO_FEdcfhi5GrHVqpuY3fR-N9ny__-ku8BIOzFEpDytCVN6Iw9rrtTnGvbNFG6fEarM',
 'scope': 'user-read-recently-played',
 'expires_at': 1590359901}

In [173]:
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [174]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [175]:
mallmisery_id = get_id('The Mall & Misery', token)
print(mallmisery_id)

2ZgJ3fxDrGSi3VKe84YDAe


In [176]:
mallmisery_features = get_features(mallmisery_id, token)
mallmisery_features

{'danceability': 0.585,
 'energy': 0.653,
 'key': 3,
 'loudness': -6.473,
 'mode': 0,
 'speechiness': 0.0377,
 'acousticness': 0.364,
 'instrumentalness': 0.0117,
 'liveness': 0.133,
 'valence': 0.284,
 'tempo': 136.937,
 'type': 'audio_features',
 'id': '2ZgJ3fxDrGSi3VKe84YDAe',
 'uri': 'spotify:track:2ZgJ3fxDrGSi3VKe84YDAe',
 'track_href': 'https://api.spotify.com/v1/tracks/2ZgJ3fxDrGSi3VKe84YDAe',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2ZgJ3fxDrGSi3VKe84YDAe',
 'duration_ms': 246973,
 'time_signature': 4}

In [177]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName']
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})

In [178]:
df = pd.DataFrame(with_features)
df.to_csv('streaming_history.csv')